In [27]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
from pathlib import Path
from tbparse import SummaryReader


In [28]:
MODEL_TYPE='GRU' # or informer

SAVE_PATH = f"../results/{MODEL_TYPE}_results.md"
SEED=21
OVERWRITE=True
if OVERWRITE:
     open(SAVE_PATH, 'w').close()

In [29]:
def get_metric_results_from_eventfile(dirpath, metrics, SAVE_PATH="", title=""):
    reader = SummaryReader(dirpath, pivot=False, extra_columns={'dir_name'})
    df = reader.scalars
    df = df.rename(columns={"tag": "metric", "dir_name": "fold"})
    df = df.groupby(["fold","metric"])['value'].max().reset_index()
    df = df[df["metric"].isin(metrics)].set_index(["fold"]).sort_values("metric").reset_index()
    res = df.pivot(index='fold', columns='metric', values='value').rename_axis(columns=None)
    if SAVE_PATH:
        res.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True,floatfmt='.3f')
    return res

In [30]:
labels = ['Rote_X', 'Rote_Y', "Rote_Z",
          'Inference_X', "Inference_Y","Inference_Z",
          "Deep_X", "Deep_Z",
          "MW"]

In [31]:
metrics = ["train_auroc", "val_auroc", "train_accuracy_epoch", "val_accuracy_epoch"]
val_aurocs = []
for label in labels:
    tit=f"# {label}"
    with open(SAVE_PATH, "a") as f:
        f.write("\n\n")
        f.write(tit)
        f.write("\n")
    res=get_metric_results_from_eventfile(f"../lightning_logs/{MODEL_TYPE}_{label}", metrics, SAVE_PATH)
    res['label']=label
    val_aurocs.append({"label":label, 
                       "mean AUROC": res[['val_auroc']].mean().item(),
                       "min AUROC": res[['val_auroc']].min().item(),
                       "max AUROC": res[['val_auroc']].max().item()
                       })

ValueError: File or directory not found: ../lightning_logs/GRU_Rote_X

In [ ]:
tit="Held-out AUROCs"
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(val_aurocs)
tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')


In [32]:
# Get pretraining task val metrics 
tasks=['fi','cl','pc','rc']
metric_types=['FI: AUROC','CL: Accuracy','PC: MSE (deg)','RC: MSE (deg)']
varname = 'val_{task}_metric'
metrics=[varname.format(**locals()) for task in tasks]
res=get_metric_results_from_eventfile(f"../lightning_logs/{MODEL_TYPE}_pretraining_seed{SEED}", metrics, SAVE_PATH)

pretrain_summary=[]
for metric in metrics:
    pretrain_summary.append({"metric":metric, 
                       "mean": res[[metric]].mean().item(),
                       "min": res[[metric]].min().item(),
                       "max": res[[metric]].max().item()
                       })
    
tit="Pretraining validation metrics"
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(pretrain_summary)
tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')
